In [1]:
#collabrativefiltering 
import pandas as pd 
import numpy as np 

In [3]:
movies= pd.read_csv(r'D:\movies.csv', usecols=['movieId','title'],dtype={'movieId':'int32','title': 'str'})
rating = pd.read_csv(r'D:\ratings.csv', usecols=['userId', 'movieId','rating'],
                    dtype={'userId':'int32','movieId':'int32','rating':'float32'})

In [4]:
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [5]:
rating.head()

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5


In [6]:
df=pd.merge(movies,rating,on='movieId')
df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),4,4.0
1,1,Toy Story (1995),10,5.0
2,1,Toy Story (1995),14,4.5
3,1,Toy Story (1995),15,4.0
4,1,Toy Story (1995),22,4.0


In [7]:
combine_movie_rating=df.dropna(axis=0,subset=['title'])
movie_ratingcount=(combine_movie_rating.groupby(by=['title'])['rating'].
                   count().
                   reset_index().
                   rename(columns={'rating':'totalratingcount'})
                   [['title','totalratingcount']]
                  )
movie_ratingcount.head()

,title,totalratingcount
0,"""Great Performances"" Cats (1998)",225
1,#1 Cheerleader Camp (2010),9
2,#Captured (2017),1
3,#Horror (2015),19
4,#SCREAMERS (2016),1


In [8]:
rating_totalratingcount= combine_movie_rating.merge(movie_ratingcount, left_on='title', right_on='title', how='left')
rating_totalratingcount.head()

,movieId,title,userId,rating,totalratingcount
0,1,Toy Story (1995),4,4.0,68469
1,1,Toy Story (1995),10,5.0,68469
2,1,Toy Story (1995),14,4.5,68469
3,1,Toy Story (1995),15,4.0,68469
4,1,Toy Story (1995),22,4.0,68469


In [9]:
pd.set_option('display.float_format', lambda x:'%.3f' %x)
print(movie_ratingcount['totalratingcount'].describe() )

count   53817.000
mean      515.700
std      2936.711
min         1.000
25%         2.000
50%         7.000
75%        48.000
max     97999.000
Name: totalratingcount, dtype: float64


In [10]:
popularity_threshold=20000
rating_popular_movie=rating_totalratingcount.query('totalratingcount >= @popularity_threshold')
rating_popular_movie.head()

,movieId,title,userId,rating,totalratingcount
0,1,Toy Story (1995),4,4.000,68469
1,1,Toy Story (1995),10,5.000,68469
2,1,Toy Story (1995),14,4.500,68469
3,1,Toy Story (1995),15,4.000,68469
4,1,Toy Story (1995),22,4.000,68469


In [11]:
rating_popular_movie.shape

(8202258, 5)

In [12]:
#creating a pivot matrix
movie_feature=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_feature.head()

userId,3,4,5,6,8,9,10,11,12,14,...,283218,283219,283220,283221,283222,283223,283224,283225,283226,283228
title,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey (1968),0.000,4.000,0.000,0.000,0.000,0.000,5.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,5.000,0.000,0.000,0.000
Ace Ventura: Pet Detective (1994),0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000,3.000,0.000
Ace Ventura: When Nature Calls (1995),0.000,2.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,2.000,0.000,1.000,0.000,0.000,0.000
Addams Family Values (1993),0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Airplane! (1980),0.000,4.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,5.000


In [15]:
from scipy.sparse import csr_matrix

movie_feature_matrix = csr_matrix(movie_feature.values)

from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm = 'brute')
model_knn.fit(movie_feature_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [16]:
movie_feature.shape

(238, 246816)

In [20]:
query_index = np.random.choice(movie_feature.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(movie_feature.iloc[query_index,:].values.reshape(1,-1,), n_neighbors = 6)

67


In [22]:
for i in range (0, len(distances.flatten())):
    if i == 0:
        print('similar movies like {0}:\n'.format(movie_feature.index[query_index]))
    else:
        print('{0}:{1} with the distance of {2}'.format(i,movie_feature.index[indices.flatten()[i]],distances.flatten()[i]))

similar movies like Demolition Man (1993):

1:Speed (1994) with the distance of 0.5334093570709229
2:Terminator 2: Judgment Day (1991) with the distance of 0.5350672006607056
3:Stargate (1994) with the distance of 0.5368087291717529
4:True Lies (1994) with the distance of 0.550707995891571
5:Die Hard: With a Vengeance (1995) with the distance of 0.5524563789367676
